In [4]:
import os, platform, time, urllib.request, openpyxl, operator
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from openpyxl import Workbook
from selenium import webdriver

In [6]:
class InstagramScrapper:
    def Create_Dir(self, dir_name):
        if not os.path.exists("data"):
            try:
                os.mkdir("data")
                print("Created directory 'data'")
            except:
                print("Unable to create directory 'data': Directory already exists")
        else:
            print("Unable to create directory 'data': Directory already exists")

        if not os.path.exists("data/data_" + dir_name):
            try:
                os.mkdir("data/data_" + dir_name)
                print("Created directory 'data/data_" + dir_name + "'")
            except:
                print("Unable to create directory 'data/data_" + dir_name + "': Directory already exists")
        else:
            print("Unable to create directory 'data/data_" + dir_name + "': Directory already exists")

        if not os.path.exists("data/data_" + dir_name + '/img'):
            try:
                os.mkdir("data/data_" + dir_name + '/img')
                print("Created directory 'data/data_" + dir_name + "/img'")
            except:
                print("Unable to create directory 'data/data_" + dir_name + "/img': Directory already exists")
        else:
            print("Unable to create directory 'data/data_" + dir_name + "/img': Directory already exists")

    def Scrape_Instagram(self, tag, limit=20, browser='chrome'):
        self.Create_Dir(tag)

        print("Starting Scrapping Instagram")
        file_path = "data/data_" + tag
        keyword = tag
        # Adding path.
        if not os.getcwd() in os.get_exec_path():
            # print('adding path')
            if platform.system() == "Windows":
                os.environ["PATH"] = os.environ["PATH"] + ";" + os.getcwd()
            else:
                os.environ["PATH"] = os.environ["PATH"] + ":" + os.getcwd()

        # opening instagram in browser
        
        #if 'chrome' in browser.lower():
        #    driver = webdriver.Chrome()
        #else:
        #    driver = webdriver.Firefox()
        
        driver = webdriver.Chrome('../chromedriver')
        driver.get("https://www.instagram.com/" + "explore/tags/" + tag)

        print("Loading Posts")
        time.sleep(10)
        print("Loading Data")

        # Clicking on load more once to load more images. Afterwards we will just
        # tap space to scroll to the page end to load more images
        actions = ActionChains(driver)
        actions.send_keys(Keys.SPACE).perform()
        actions.send_keys(Keys.SPACE).perform()
        actions.send_keys(Keys.SPACE).perform()

        time.sleep(5)

        # Just tap space to scroll to the page end to load more images
        clear = lambda: os.system('cls')
        msg = "Loading Images"
        class_div_img = ["_si7dy"]
        for div in class_div_img:
            if len(driver.find_elements_by_class_name(div)) > 1:
                while (len(driver.find_elements_by_class_name(div)) ) <= limit :
                    actions.send_keys(Keys.SPACE).perform()
                    msg = msg + "."
                    print(msg)
                    print(len(driver.find_elements_by_class_name(div)))
                    time.sleep(2.5)
                    if len(msg) > 18:
                        msg = "Loading Images"
        print(str(limit) + " images loaded")

        # Storing images links and captions. Slicing with 9 because first 9 posts
        # are the 'most popular' posts for that tag
        img_src = []
        img_caption = []
        hashtags = {}

        for data in driver.find_elements_by_class_name("FFVAD"):
            #     u.get_attribute("href").split("/")[4]
            img_caption.append(data.get_attribute("alt"))
            img_src.append(data.get_attribute("src"))

        img_caption = img_caption[9:limit + 9]
        img_src = img_src[9:limit + 9]
        img_caption.sort()

        # Create a workbook for excel
        tag_File = file_path + "/" + tag + "_Instagram.xlsx"
        wb = openpyxl.Workbook()
        ws_Captions = wb.create_sheet(title="Caption")
        col = 'A'
        row = 1

        print("Dumping data in excel file")
        for caption in img_caption:
            tags = caption.split("#")
            # write caption to excel file
            ws_Captions[col + str(row)] = tags[0]
            row += 1

            # strip tags
            tags = tags[1:]
            for tag in tags:
                cleaned = tag.replace(" ", "").replace("\n", "")
                cleaned = cleaned.lower()
                if cleaned not in hashtags and len(cleaned) < 20:
                    hashtags[cleaned] = 1
                elif cleaned in hashtags and len(cleaned) < 20:
                    hashtags[cleaned] = hashtags[cleaned] + 1

        # sort hashtags with frequencies and store them in excel
        hashtags = sorted(hashtags.items(), key=operator.itemgetter(1), reverse=True)

        ws_Tags = wb.create_sheet(title="Tags")
        tagName = 'A'
        tagFreq = 'B'
        row = 1

        for tag in hashtags:
            ws_Tags[tagName + str(row)] = tag[0]
            ws_Tags[tagFreq + str(row)] = tag[1]
            row += 1

        wb.save(tag_File)

        print("Dumping Images. This will take some time!")
        row = 1
        for src in img_src:
            urllib.request.urlretrieve(src, file_path + '/img/Instagram_' + str(row) + ".jpeg")
            row += 1
            if (row % 10 == 0):
                print("(" + str(row) + "/" + str(len(img_src)) + ") Images Downloaded")

        print("Closing Instagram")
        driver.quit()

In [9]:
#from src.InstagramScrapper import InstagramScrapper
#from src.TwitterScrapper import TwitterScrapper

if __name__ == '__main__':

    print("Starting Social Media Scrapper...")
    keyword = str(input("Enter keyword to search for: "))
    insta_limit = int(input("Enter how many posts to scrape from Instagram: "))
    #twitter_limit = int(input("Enter how many posts to scrape from Twitter: "))

    consumerKey = 'Y2x9rnSVirom9p4aP4j4GmWGy'
    consumerSecret = 'peCyxeqB68YvMrcdjXilHbzbWe3KXZKD4cE6NzHnjxTd2GCm9u'
    accessToken = '2186053585-zX6VlzWtTr9nNg72SXk9q0TWe6yV6VDyI0TCaxF'
    accessTokenSecret = 'T3NAV6vXeOzHXLwBRulUXyBxRQUP8cjdbepFkeFzQyMgh'


    scrapper = InstagramScrapper()
    driver = webdriver.Chrome('../chromedriver')
    scrapper.Scrape_Instagram(tag=keyword,
                              limit=insta_limit,
                              browser=driver) # 'chrome' or 'firefox'


    #twitter = TwitterScrapper()
    #twitter.Scrape_Twitter(Consumer_Key=consumerKey,
    #                       Consumer_Secret=consumerSecret,
    #                       Access_Token=accessToken,
    #                       Access_Token_Secret=accessTokenSecret,
    #                       tag=keyword,
    #                       limit=twitter_limit,
    #                       lang='en')  # Language codes: https://en.wikipedia.org/wiki/List_of_ISO_639-1_codes


    print("Stopping Social Media Scrapper...")

Starting Social Media Scrapper...


Enter keyword to search for:  espaciosya
Enter how many posts to scrape from Instagram:  10
Enter how many posts to scrape from Twitter:  0


Unable to create directory 'data': Directory already exists
Created directory 'data/data_espaciosya'
Created directory 'data/data_espaciosya/img'
Starting Scrapping Instagram
Loading Posts
Loading Data
10 images loaded
Dumping data in excel file
Dumping Images. This will take some time!
Closing Instagram
Stopping Social Media Scrapper...


In [11]:
hashtags

NameError: name 'hashtags' is not defined